In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
x_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')
x_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

In [3]:
print(x_train.shape)
print(x_test.shape)

(300, 224, 224, 3)
(30, 224, 224, 3)


In [4]:
print(y_train.shape)
print(y_test.shape)

(300,)
(30,)


In [5]:
#x_train /= 255????

In [6]:
from __future__ import print_function

import numpy as np
from random import shuffle
import time
import csv
from PIL import Image
import os
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import initializers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import *
from keras.callbacks import CSVLogger
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics as sklm
import sys
sys.path.insert(0,'..')
from utils import lossprettifier
from Classifier.VGG import *

C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\f

In [7]:
# for reproducibility
np.random.seed(3768)

# use this environment flag to change which GPU to use 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used

#Get TensorFlow session
def get_session(): 
  config = tf.ConfigProto() 
  config.gpu_options.allow_growth = True 
  return tf.Session(config=config) 
  
# One hot encoding of labels 
def dense_to_one_hot(labels_dense,num_clases=4):
  return np.eye(num_clases)[labels_dense]

In [8]:
# Preparing training and test sets
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [9]:
y_train = dense_to_one_hot(y_train,num_clases=4)
y_valid= dense_to_one_hot(y_valid,num_clases=4)

In [10]:
#Image data generation for the training 
datagen = ImageDataGenerator(
               featurewise_center = False, 
               samplewise_center = False,  # set each sample mean to 0
               featurewise_std_normalization = True,  
               samplewise_std_normalization = False)  

datagen.fit(x_train) 
for i in range(len(x_test)):
      x_test[i] = datagen.standardize(x_test[i])

C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\keras_preprocessing\image\image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [11]:
#Defining hyperparameters
batch_Size = 32
steps_Per_Epoch = 32
numEpochs = 50

#Instantating VGG19 model
model = VGG19((224,224,3),4) #VGG19_dense for revised VGG19, VGG19 for VGG19. Please pay attention to VGG16(), chnage the input shape and class number in VGG.py.

#Creating an optimizers
adaDelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.95, nesterov=True)
model.compile(optimizer = sgd , loss = 'categorical_crossentropy', metrics = ['accuracy'])

#Creating early stopping 
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 50, verbose = 1, mode = 'auto', restore_best_weights = True)       

train_generator = datagen.flow(x_train, y_train, batch_size = batch_Size)
validation_generator = datagen.flow(x_valid, y_valid, batch_size = batch_Size)

# Model training
history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_Per_Epoch,
    validation_data = validation_generator, 
    validation_steps = 16,
    epochs = numEpochs,
    shuffle = True, 
    verbose = 1)



Epoch 1/50
32/32 [==============================] - 5s 169ms/step - loss: 1.1469 - accuracy: 0.3813 - val_loss: 1.0752 - val_accuracy: 0.5333
Epoch 2/50
32/32 [==============================] - 3s 93ms/step - loss: 0.9644 - accuracy: 0.5103 - val_loss: 1.0205 - val_accuracy: 0.5333
Epoch 3/50
32/32 [==============================] - 3s 92ms/step - loss: 0.7291 - accuracy: 0.7038 - val_loss: 1.1657 - val_accuracy: 0.6333
Epoch 4/50
32/32 [==============================] - 3s 97ms/step - loss: 0.5836 - accuracy: 0.7526 - val_loss: 1.2004 - val_accuracy: 0.7000
Epoch 5/50
32/32 [==============================] - 3s 95ms/step - loss: 0.4947 - accuracy: 0.8052 - val_loss: 1.4091 - val_accuracy: 0.6667
Epoch 6/50
32/32 [==============================] - 3s 99ms/step - loss: 0.3958 - accuracy: 0.8382 - val_loss: 1.4711 - val_accuracy: 0.6000
Epoch 7/50
32/32 [==============================] - 3s 89ms/step - loss: 0.3666 - accuracy: 0.8519 - val_loss: 1.7593 - val_accuracy: 0.6667
Epoch 8/50

In [12]:
modelPath = "VGG19_COVID19.h5"
resultPath = 'VGG19_COVID19.txt'

In [13]:
y_test_oh = dense_to_one_hot(y_test, num_clases=4)

# visualizing losses and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']

#Observing the losses but can be commented out as it's not mandatory 
reporter = lossprettifier.LossPrettifier(show_percentage=True)

for i in range(numEpochs-1):
    reporter(epoch=i, LossA = train_loss[i], LossAB = val_loss[i])

# Model evaluation 
score, acc = model.evaluate(x_test, y_test_oh, batch_size=batch_Size)
print("Accuracy:", acc)

#if acc>0.675:
model.save_weights(modelPath)

Epoch     0 | LossA: 1.15(+0.00%) 	| LossAB: 1.08(+0.00%) 	
Epoch     1 | LossA: 0.97(-15.88%) ▼	| LossAB: 1.02(-5.08%) ▼	
Epoch     2 | LossA: 0.73(-24.77%) ▼	| LossAB: 1.17(+14.23%) ▲	
Epoch     3 | LossA: 0.59(-18.56%) ▼	| LossAB: 1.20(+2.97%) ▲	
Epoch     4 | LossA: 0.49(-16.64%) ▼	| LossAB: 1.41(+17.39%) ▲	
Epoch     5 | LossA: 0.40(-18.11%) ▼	| LossAB: 1.47(+4.40%) ▲	
Epoch     6 | LossA: 0.37(-7.91%) ▼	| LossAB: 1.76(+19.59%) ▲	
Epoch     7 | LossA: 0.33(-10.20%) ▼	| LossAB: 1.37(-22.27%) ▼	
Epoch     8 | LossA: 0.34(+1.01%) ▲	| LossAB: 1.06(-22.62%) ▼	
Epoch     9 | LossA: 0.24(-30.08%) ▼	| LossAB: 1.99(+88.01%) ▲	
Epoch    10 | LossA: 0.25(+7.05%) ▲	| LossAB: 1.78(-10.55%) ▼	
Epoch    11 | LossA: 0.24(-4.13%) ▼	| LossAB: 1.54(-13.31%) ▼	
Epoch    12 | LossA: 0.25(+4.58%) ▲	| LossAB: 1.84(+19.50%) ▲	
Epoch    13 | LossA: 0.25(-0.67%) ▼	| LossAB: 1.96(+6.33%) ▲	
Epoch    14 | LossA: 0.23(-7.79%) ▼	| LossAB: 2.65(+35.31%) ▲	
Epoch    15 | LossA: 0.25(+9.49%) ▲	| LossAB: 3.10(+16.

In [14]:
y_pred = model.predict(x_test)
y_pred = y_pred.reshape(len(y_test), 4)
y_pred = np.argmax(y_pred, axis=1)

# Writing results on file
f = open(resultPath,'a') #create classification report
f.write(classification_report(y_test, y_pred))
f.write(str(sklm.cohen_kappa_score(y_test, y_pred))+","+str(acc)+","+str(score)+"\n")

#Print class-wise classification metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.60      0.67        10
           1       0.57      0.80      0.67        10
           2       0.38      0.30      0.33        10

    accuracy                           0.57        30
   macro avg       0.57      0.57      0.56        30
weighted avg       0.57      0.57      0.56        30



In [15]:
f.close()